In [918]:
import pandas as pd
import numpy as np
from datetime import datetime
import psycopg2

### Extract data from CSV files - Extract

In [919]:
employees_df = pd.read_csv("../data/employee_data.csv")
training_development_df = pd.read_csv("../data/training_and_development_data.csv")
employees_engagment_df = pd.read_csv("../data/employee_engagement_survey_data.csv")

##### Function to Create/Connect in database

In [920]:
def create_database():
  try:
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=--")
    
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    cur.execute(f"DROP DATABASE IF EXISTS employees")
    cur.execute(f"CREATE DATABASE employees")
  except psycopg2.Error as e:
    print(e)
  finally:
    conn.close()

  conn = psycopg2.connect(f"host=localhost dbname=employees user=postgres password=--")
  cur = conn.cursor()

  return cur, conn

In [922]:
cur, conn = create_database()

database "employees" is being accessed by other users
DETAIL:  There are 7 other sessions using the database.



### Transform

In [923]:
employees_df = employees_df.loc[:, ["EmpID","FirstName","LastName","DOB","GenderCode","State","RaceDesc","StartDate","ExitDate",\
  "Title","EmployeeStatus","EmployeeType","EmployeeClassificationType","TerminationType","DepartmentType","Division","JobFunctionDescription",\
  "Performance Score","Current Employee Rating"]]

employees_df.rename(columns={"DOB":"Birthday"}, inplace=True)

for i, row in employees_df.iterrows():
  date = row['Birthday']
  d, m, y = date.split("-")
  employees_df.iloc[i, 3] = f"{m}-{d}-{y}"

for i, row in employees_df.iterrows():
  date_obj = datetime.strptime(row['StartDate'], "%d-%b-%y")
  m = date_obj.month  
  d, _, y = date.split("-")
  employees_df.iloc[i, 7] = f"{m}-{d}-{y}"

for i, row in employees_df.iterrows():
  if(row['ExitDate'] is not np.nan):
     date_obj = datetime.strptime(row['ExitDate'], "%d-%b-%y")
     m = date_obj.month  
     d, _, y = date.split("-")
     employees_df.iloc[i, 8] = f"{m}-{d}-{y}"
  else:
    pass

In [924]:
training_development_df.rename(columns={"Employee ID": "Employee_ID", "Training Date": "Training_Date", "Training Program Name": "Training_Program_Name	",\
  "Training Type": "Training_Type","Training Outcome": "Training_Outcome", "Training Duration(Days)": "Training_Duration(Days)", "Training Cost": "Training_Cost"}, inplace=True)

for i, row in training_development_df.iterrows():
  date_obj = datetime.strptime(row['Training_Date'], "%d-%b-%y")
  m = date_obj.month  
  d, _, y = date.split("-")
  training_development_df.iloc[i, 1] = f"{m}-{d}-{y}"

In [925]:
employees_engagment_df.rename(columns={"Employee ID": "Employee_ID", "Survey Date": "Survey_Date", "Engagement Score": "Engagement_Score","Satisfaction Score":"Satisfaction_Score","Work-Life Balance Score": "Work-Life_Balance_Score"}, inplace=True)

for i, row in employees_engagment_df.iterrows():
  date = row['Survey_Date']
  d, m, y = date.split("-")
  employees_engagment_df.iloc[i, 1] = f"{m}-{d}-{y}"


### Create tables definition for the three DataFrames

##### Data Model
![Data Model](../img/Employees.png)

In [926]:
employees_table_create = ("""CREATE TABLE IF NOT EXISTS employees_data(
    Emp_ID INT PRIMARY KEY,
    First_Name VARCHAR(60) NOT NULL,
    Last_Name VARCHAR(60) NOT NULL,
    Birthday VARCHAR(10) NOT NULL,
    Gender_Code VARCHAR(6) NOT NULL,
    State VARCHAR(2) NOT NULL,
    Race_Desc VARCHAR(10) NOT NULL,
    Start_Date VARCHAR(10) NOT NULL,
    Exit_Date VARCHAR(10),
    Title VARCHAR(3000) NOT NULL,
    Employee_Status VARCHAR(30) NOT NULL,
    Employee_Type VARCHAR(30) NOT NULL,
    Employee_ClassificationType VARCHAR(30) NOT NULL,
    Termination_Type VARCHAR(30) NOT NULL,
    Department_Type VARCHAR(30) NOT NULL,
    Division VARCHAR(60) NOT NULL,
    Job_Function_Description VARCHAR(60) NOT NULL,
    Performance_Score VARCHAR(60) NOT NULL,
    Current_Employee_Rating INT
)""")

cur.execute(employees_table_create)
conn.commit()

In [927]:
training_development_create_table = ("""CREATE TABLE IF NOT EXISTS training_data (
    Employee_ID INT PRIMARY KEY,
    Training_Date VARCHAR(10) NOT NULL,
    Training_Program_Name varchar(100) NOT NULL,
    Training_Type VARCHAR(100) NOT NULL,
    Training_Outcome VARCHAR(100) NOT NULL,
    Location VARCHAR(100) NOT NULL,
    Trainer VARCHAR(100) NOT NULL,
    Training_Duration_Days INT NOT NULL,
    Training_Cost numeric(10, 2) NOT NULL
)""")

cur.execute(training_development_create_table)
conn.commit()

In [928]:
employees_engagment_df_create_table = ("""CREATE TABLE IF NOT EXISTS employees_survey(
    Employee_ID INT PRIMARY KEY,
    Survey_Date VARCHAR(10) NOT NULL,
    Engagement_Score INT NOT NULL,
    Satisfaction_Score INT NOT NULL,
    WorkLife_Balance_Score INT NOT NULL
)""")

cur.execute(employees_engagment_df_create_table)
conn.commit()

### Dataframe data to PostgresSQL - Loading

In [929]:
employees_table_insert = ("""INSERT INTO employees_data(
            emp_id, first_name, last_name, birthday, gender_code, state, race_desc,
            start_date, exit_date, title, employee_status, employee_type,
            employee_classificationType, termination_type, department_type, division,
            job_function_description, performance_score, current_employee_rating
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s, %s
        )
    """)

In [930]:
for i, row in employees_df.iterrows():
  cur.execute(employees_table_insert, tuple(row))

In [931]:
training_development_create_table = ("""INSERT INTO training_data (
    Employee_ID,
    Training_Date,
    Training_Program_Name,
    Training_Type,
    Training_Outcome,
    Location,
    Trainer,
    Training_Duration_Days,
    Training_Cost) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
""")

In [932]:
for i, row in training_development_df.iterrows():
  cur.execute(training_development_create_table, tuple(row))

In [933]:
employees_engagment_insert = ("""INSERT INTO employees_survey (
  Employee_ID, 
  Survey_Date, 
  Engagement_Score, 
  Satisfaction_Score, 
  WorkLife_Balance_Score) 
  VALUES(%s, %s, %s, %s, %s)
""")

In [934]:
for i, row in employees_engagment_df.iterrows():
  cur.execute(employees_engagment_insert, tuple(row))

In [935]:
conn.commit()